# FastWoe Monotonic Constraints

Author: https://www.github.com/xRiskLab

In [11]:
from pathlib import Path

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from fastwoe import FastWoe

In [12]:
# Set the path to the data directory
data_dir = Path.cwd().parent / "data"

# Load the data
df = pd.read_csv(data_dir / "BankCaseStudyData.csv")

# Define the features and label
features = [
    "Application_Score",
    "Bureau_Score",
    "Number_of_Payments",
]

label = "Final_Decision"

X = df[features]
y = df[label].map({"Accept": 0, "Decline": 1})

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

pipeline = Pipeline(
    [
        (
            "woe",
            FastWoe(
                binning_method="tree",
                tree_kwargs={"max_depth": 2, "min_samples_leaf": 5},
                monotonic_cst={
                    "Application_Score": -1,
                    "Bureau_Score": -1,
                    "Number_of_Payments": 1,
                },
                random_state=42,
            ),
        ),
        ("logistic_regression", LogisticRegression()),
    ]
)

pipeline.fit(X_train, y_train)

Pipeline(steps=[('woe', <fastwoe.fastwoe.FastWoe object at 0x1318d3190>),
                ('logistic_regression', LogisticRegression())])

In [13]:
# Check the WOE mappings to verify monotonic constraints
woe = pipeline.named_steps["woe"]

print("Application_Score (constraint: -1, WOE should decrease):")
print(woe.get_mapping("Application_Score"))
print("\nBureau_Score (constraint: -1, WOE should decrease):")
print(woe.get_mapping("Bureau_Score"))
print("\nNumber_of_Payments (constraint: 1, WOE should increase):")
print(woe.get_mapping("Number_of_Payments"))

Application_Score (constraint: -1, WOE should decrease):
         category  count  count_pct  good_count  bad_count  event_rate  \
0     (-∞, 777.5]   1878   9.443355         575       1303    0.693823   
1  (777.5, 807.5]    314   1.578921         176        138    0.439490   
2  (807.5, 892.5]   2903  14.597476        2507        396    0.136411   
3      (892.5, ∞)  14792  74.380248       14551        241    0.016293   

        woe    woe_se  woe_ci_lower  woe_ci_upper  
0  2.966353  0.050066      2.868225      3.064480  
1  1.905068  0.113702      1.682215      2.127920  
2  0.302870  0.054075      0.196885      0.408856  
3 -1.952320  0.064947     -2.079614     -1.825026  

Bureau_Score (constraint: -1, WOE should decrease):
         category  count  count_pct  good_count  bad_count  event_rate  \
0     (-∞, 801.5]   1866   9.383014         544       1322    0.708467   
1  (801.5, 808.5]    288   1.448182         166        122    0.423611   
2  (808.5, 822.5]    656   3.298637  